In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/통합데이터/필터된 데이터.csv")

In [3]:
# 2. 결측치 처리 (안전장치)
df['증감율_이용금액_신용_전월'] = df['증감율_이용금액_신용_전월'].fillna(0)
df['연체잔액_B0M'] = df['연체잔액_B0M'].fillna(0)

# 3. 이탈 위험군 조건 정의 (4가지)
cond_status = (df['회원여부_이용가능'] == 0)
cond_inactive = (df['연속무실적개월수_기본_24M_카드'] >= 2)
cond_drop = (df['증감율_이용금액_신용_전월'] <= -0.3)
cond_overdue = (df['연체잔액_B0M'] > 0)

# 4. [핵심] 옆에 피쳐 하나 만들기! (Churn_Refined)
df['Churn_Refined'] = np.where(cond_status | cond_inactive | cond_drop | cond_overdue, 1, 0)

# 5. 결과 확인 (눈으로 직접 보기)
print("새로 생긴 컬럼 확인:")
print(df[['발급회원번호', 'Churn_Refined']].head()) # 회원번호 옆에 딱 붙었는지 확인!

새로 생긴 컬럼 확인:
  발급회원번호  Churn_Refined
0  SYN_0              0
1  SYN_1              1
2  SYN_2              1
3  SYN_3              0
4  SYN_4              1


In [ ]:
df.head()